In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.reporting import reports as rp
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
mv_inputs_dict = dm.get_mv_inputs_data('inputs_test.xlsx')
mv_inputs = summary.get_mv_inputs(mv_inputs_dict)
mv_outputs = summary.get_mv_output(mv_inputs, 'Equity')
rp.get_output_report('output_from_mv', mv_outputs)

In [ ]:
ts_dict = dm.get_ts_data('return_data.xlsx')
ts_outputs = summary.get_ts_output(ts_dict)
rp.get_output_report('output_from_ts', ts_outputs)

In [ ]:
#create plan object that has the relevant variables (cov, policy ret, var,fsv )
summarized_output = dm.get_output_data('output.xlsx')
plan = summary.get_plan_params(summarized_output)
plan.symbols

In [ ]:
bnds = dm.get_bounds(dm.OUTPUTS_FP+'bounds.xlsx')
bnds

In [ ]:
cons = ({'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1]) - x[3] - .02})

In [ ]:
# Optimizing for maximum sharpe ratio
opt_sharpe = plan.optimize(plan.min_sharpe_ratio, bnds, cons)
opt_sharpe

In [ ]:
# Optimizing for minimum variance
opt_var = plan.optimize(plan.min_variance,bnds, cons)
# opt_var
min_ret = plan.portfolio_stats(opt_var['x'])[0]
min_ret

In [ ]:
# Optimizing for max return
opt_ret = plan.optimize(plan.min_ret,bnds, cons)
# opt_ret
max_ret = np.around(plan.portfolio_stats(opt_ret['x']),4)[0]
max_ret

In [ ]:
#Getting data for efficient frontier portfolios
target_rets = np.linspace(min_ret,max_ret,100)
# target_rets = np.linspace(-.01,0.04,100)
t_vols = []
t_weights = []

for tr in target_rets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: plan.portfolio_stats(x)[0] - tr},
               {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1])-x[3] - .02})
    
    opt_ef = plan.optimize(plan.min_volatility, bnds, ef_cons)
    
    t_vols.append(opt_ef['fun'])
    t_weights.append(opt_ef['x'])
    
target_vols = np.array(t_vols)
target_weights = np.array(t_weights)
optimized_weights = np.transpose(target_weights)
# opt_ef

In [ ]:
#Dataframe containing efficient frontier portfolios (return, vol, sharpe and weights)
ports_df = dm.get_ports_df(target_rets, target_vols, target_weights, plan.symbols, raw=False)
ports_df

In [ ]:
#Asset Allocation Plot
combo_list = [('15+ STRIPS','forestgreen'),('Long Corporate','lightgreen'), ('Ultra 30-Year UST Futures','greenyellow'),
              ('Equity','deeppink'),('Liquid Alternatives','blue'),
              ('Private Equity','darkred'),('Credit','yellow'),('Real Estate','orange'),
              ('Equity Hedges','blueviolet'),('Cash','khaki')]

fig = go.Figure()

for combo in combo_list:
    fig.add_trace(go.Scatter(
     x= ports_df['Return'], y = ports_df[combo[0]],
     name = combo[0],
     mode = 'lines',
     line=dict(width=0.5, color=combo[1]),
     stackgroup = 'one'))
fig.update_layout(
        title = {'text':"<b>Asset Allocation</b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(ticksuffix="%"),
        yaxis = dict(ticksuffix="%"),
        legend_title="<b>Assets</b>",
        plot_bgcolor='White'
    )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Returns</b>",
    title_font = {"size": 20},showline=True,linewidth=2,linecolor='black',mirror=False)
fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Weights</b>",range=(0,162),title_font = {"size": 20},
    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.show()

In [ ]:
#Plotly version of the Efficient Frontier plot

fig = px.scatter(ports_df, x="Volatility", y="Return",color='Sharpe')
fig.update_layout(
    title={
            'text': "<b>Efficient Frontier Portfolio</b>",
            'y':0.9,
            'x':0.5,'xanchor': 'center',
            'yanchor': 'top'
            },
    title_font_family="Calibri",
    titlefont = {"size":20},
        xaxis = dict(tickfont = dict(size=14),ticksuffix="%"),
       yaxis = dict(ticksuffix="%"),
#     legend_title="Strategies",
    showlegend=True,
    plot_bgcolor='White'
                 )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Expected Volatility</b>",title_font = {"size": 20},
                    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Expected Return</b>",title_font = {"size": 20},
                 showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

In [ ]:
#Export Efficient Frontier portfoio data to excel
ports_df_2 = dm.get_ports_df(target_rets, target_vols, target_weights, plan.symbols)
rp.get_ef_portfolios_report('et_portolios', ports_df_2)